In [119]:
from pulp import *
from itertools import chain, combinations
import time
%run pravljenjeGrafova.ipynb
import networkx as nx

def dominant_vertices(graph):
    G = nx.Graph()
    for edge, value in graph.items():
        if value == 1:
            u, v = edge
            G.add_edge(u, v)

    ds = nx.dominating_set(G)

    return list(ds)


# funkcija za podskupove
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


# funkcija za susjede
def neighbors(x):
    n = []
    
    for (i,j) in Eweak:
        if i==x and Eweak[i,j] == 1:
            n.append(j)
    n.append(x)
    return n


# susjedstvo
def adjacency(graph):
    adjacency_dict = {}
    
    for vertex, neighbors in graph.items():
        for neighbor in neighbors:
            if int(vertex)!=int(neighbor):
                adjacency_dict[(int(vertex), int(neighbor))] = 1
    
    for vertex in graph:
        for other_vertex in graph:
            if (int(vertex), int(other_vertex)) not in adjacency_dict and int(vertex)!=int(other_vertex) :
                adjacency_dict[(int(vertex), int(other_vertex))] = 0
    
    return adjacency_dict


            
    

In [120]:
g = napraviGraf("veoma mali grafovi/graf-11.txt")

dictE= adjacency(g)
Eweak = dict(sorted(dictE.items()))

V=[*range(1,len(g)+1,1)]

dom=dominant_vertices(Eweak)

print(dom)


[1, 2, 3, 5, 6, 8, 9, 10]


In [121]:

# model - problem minimizacije
startTime = time.time()
model = LpProblem("MinimumWeaklyConnectedDominatingSet", LpMinimize)

# varijable
x = LpVariable.dicts("x", Eweak, 0, 1, LpBinary)
y = LpVariable.dicts("y", V , 0, 1, LpBinary)

# funkcija cilja
model += 1 + lpSum( x [ (u, v) ]  for (u, v) in Eweak )

# ogranicenja 1, 2 i 3
for u in V:
    for v in V:
        if u!=v:
            model += x[ (u, v) ] <= Eweak[u,v]
            model += x[ (u, v) ] == x[ (v, u) ]
            model += y[u] + y[v] >= 2*x[ (u, v) ]
          

# ogranicenje 4
for S in powerset(V):
    if len(S)>1:
        model += lpSum( x[ (u, v) ] for u in S for v in S if u!=v) <= len(S) - 1
        #sum1=0
        #for u in V:
         #    for v in V:
          #      if u!=v:
           #        sum1 += x[(u, v)]
        #model += sum1 <= len(S) - 1
    
    
#ogranicenje 5
model += lpSum( x[ (u,v) ] for u in V for v in V if u!=v) == lpSum( y[t] for t in V) - 1


#ogranicenje 6

# posmatramo N[V] kao dom(V), Optimal, rezultat netacan
#model += lpSum( y[t] for t in dom) >= 1

# Infeasible, ali dobar rezultat
#for v in V:
 #   suma3=0
  #  for t in neighbors(v):
   #     suma3+=y[t]
    #model += suma3 >= 1   

# Infeasible, ali dobar rezultat
for v in V:
    model += lpSum( y[t] for t in neighbors(v)) >= 1


# rjesavanje
model.solve(PULP_CBC_CMD(maxSeconds=600, msg=True, fracGap=0))

# ispis DS
selected = [u for u, var in y.items() if var.varValue==1]

print("Odabrani:",selected)
print("Duzina:", len(selected))
print(LpStatus[model.status])

print("--- %s sekundi ---" % (time.time() - startTime))





Odabrani: [4, 7, 11]
Duzina: 3
Infeasible
--- 0.39113688468933105 sekundi ---
